In [5]:
import requests
from datetime import datetime, timezone, timedelta
import json
import pandas as pd

In [6]:
url = 'http://127.0.0.1:8000/locationConsensus/interactions/'
# url = "https://dsc180-decentralized-location.herokuapp.com/locationConsensus/interactions/"
data = requests.get(url).json()
data

[{'from_user': 'Alan',
  'spotted_users': 'Natahn',
  'time': '2023-02-09T01:09:39.573678Z'},
 {'from_user': 'Alan',
  'spotted_users': 'Mason',
  'time': '2023-02-09T01:09:27.975968Z'},
 {'from_user': 'Alan',
  'spotted_users': 'Alex',
  'time': '2023-02-09T01:09:25.131091Z'},
 {'from_user': 'Mason',
  'spotted_users': 'Alan',
  'time': '2023-02-09T01:09:20.260899Z'},
 {'from_user': 'Natahn',
  'spotted_users': 'Alan',
  'time': '2023-02-09T01:09:17.902631Z'},
 {'from_user': 'Alex',
  'spotted_users': 'Alan',
  'time': '2023-02-09T01:09:14.540994Z'}]

In [7]:
df = pd.DataFrame(data)
df

,from_user,spotted_users,time
0,Alan,Natahn,2023-02-09T01:09:39.573678Z
1,Alan,Mason,2023-02-09T01:09:27.975968Z
2,Alan,Alex,2023-02-09T01:09:25.131091Z
3,Mason,Alan,2023-02-09T01:09:20.260899Z
4,Natahn,Alan,2023-02-09T01:09:17.902631Z
5,Alex,Alan,2023-02-09T01:09:14.540994Z


In [8]:
# df["time"] = df["time"].str.split(".").str[0]
df["time"] = pd.to_datetime(df["time"])
df

,from_user,spotted_users,time
0,Alan,Natahn,2023-02-09 01:09:39.573678+00:00
1,Alan,Mason,2023-02-09 01:09:27.975968+00:00
2,Alan,Alex,2023-02-09 01:09:25.131091+00:00
3,Mason,Alan,2023-02-09 01:09:20.260899+00:00
4,Natahn,Alan,2023-02-09 01:09:17.902631+00:00
5,Alex,Alan,2023-02-09 01:09:14.540994+00:00


In [9]:
now = datetime.now(timezone.utc)
print(now)
past_hour = now - timedelta(hours=1)
print(past_hour)

2023-02-09 01:10:24.219303+00:00
2023-02-09 00:10:24.219303+00:00


In [10]:
current_df = df.loc[df["time"] > past_hour]
current_df

,from_user,spotted_users,time
0,Alan,Natahn,2023-02-09 01:09:39.573678+00:00
1,Alan,Mason,2023-02-09 01:09:27.975968+00:00
2,Alan,Alex,2023-02-09 01:09:25.131091+00:00
3,Mason,Alan,2023-02-09 01:09:20.260899+00:00
4,Natahn,Alan,2023-02-09 01:09:17.902631+00:00
5,Alex,Alan,2023-02-09 01:09:14.540994+00:00


In [11]:
spotted_percentage = current_df.groupby("spotted_users")["from_user"].count() / current_df["from_user"].nunique()

In [12]:
spotted_percentage

spotted_users
Alan      0.75
Alex      0.25
Mason     0.25
Natahn    0.25
Name: from_user, dtype: float64

In [13]:
cuttoff = 0.5
flagged_users = spotted_percentage[spotted_percentage > cuttoff]

In [14]:
blacklist_url = "http://127.0.0.1:8000/locationConsensus/blacklist/"

In [15]:
def update_blacklist(users):
    for user in users:
        toSend  = {'userID': user}
        out = requests.post(blacklist_url, data = toSend)

In [16]:
update_blacklist(flagged_users.index)

In [17]:
for user in flagged_users.index:
    print(user)

Alan
